In [1]:
import azureml.core
from azureml.core import Workspace, Datastore

import pandas as pd
import numpy as np

from azureml.core import Dataset

In [2]:
ws = Workspace.from_config()

All raw data stored in Azure blob container, so access thru a blob_datastore

In [3]:
blob_datastore_name='ivrdatasept' # Name of the Datastore  to workspace
container_name=os.getenv("BLOB_CONTAINER", "ivrdatasept") # Name of Azure blob container
account_name=os.getenv("BLOB_ACCOUNTNAME", "ivrdatasepton") # Storage account name
account_key=os.getenv("BLOB_ACCOUNT_KEY", "nZ12xgN95SfJTEzf+eNbnh89Ld6ZxJcMxIQ63AOVqZtwLk4ABZZkWO8BHkzT8iyZYqaIL5HQyOhkFykPg8SBAQ==") # Storage account key

blob_datastore = Datastore.register_azure_blob_container(workspace=ws, 
                                                         datastore_name=blob_datastore_name, 
                                                         container_name=container_name, 
                                                         account_name=account_name,
                                                         account_key=account_key)

In [4]:
# from azureml.core import Datastore
# from azureml.data.data_reference import DataReference
# from azureml.pipeline.core import PipelineData

# # def_blob_store = Datastore(ws, "workspaceblobstore")

# input_data = DataReference(
#        datastore=blob_datastore,
#        data_reference_name="WR_OrderProcessing_Extract_0501_1201.zip",
#        path_on_datastore="")

# output = PipelineData("output", datastore=def_blob_store)
    

In [5]:
# type(input_data)

In [6]:

# dataset = Dataset.Tabular.from_delimited_files(path = [(blob_datastore, 'WR_OrderProcessing_Extract_0501_1201.zip')])
# dataset = Dataset.Tabular.from_delimited_files(path = [(blob_datastore, 'ivrDetails_2019_11251728.csv')])


Download data files from blob store to local directory

In [7]:
# blob_datastore.download(target_path='ivr_data',
#                    prefix='AWR_OrderProcessing_Extract_0501_1201_v3.zip',
# #                    prefix='',
#                    show_progress=True)

The order processing data

In [8]:
order_extract_df = pd.read_csv('ivr_data/AWR_OrderProcessing_Extract_0501_1201_v3.zip')

/anaconda/envs/azureml_py36/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (10,14,92) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [134]:
all_order_list = order_extract_df.columns.tolist()

In [135]:
all_order_list

['ORDER_ID',
 'ORDER_CREATED_DATE',
 'ACCOUNT_NUMBER',
 'COMCAST_ACCOUNT_NUMBER',
 'FRAUDNET_SCORE',
 'TOTAL_PRICE',
 'SKU',
 'TYPE1',
 'MAKE1',
 'MODEL1',
 'PRICE1',
 'TYPE2',
 'MAKE2',
 'MODEL2',
 'PRICE2',
 'TYPE3',
 'MAKE3',
 'MODEL3',
 'PRICE3',
 'TYPE4',
 'MAKE4',
 'MODEL4',
 'PRICE4',
 'TYPE5',
 'MAKE5',
 'MODEL5',
 'PRICE5',
 'NUM_PORTIN',
 'PORT_IN_FLAG',
 'FROM_CARRIER_NAME',
 'ACNT_BILL_FNAME_MATCHES',
 'ACNT_BILL_LNAME_MATCHES',
 'ACNT_SHIP_FNAME_MATCHES',
 'ACNT_SHIP_LNAME_MATCHES',
 'SHIP_BILL_FNAME_MATCHES',
 'SHIP_BILL_LNAME_MATCHES',
 'BILL_SHIP_ADDR_MATCHES',
 'BILLING_CITY',
 'BILLING_STATE',
 'BILLING_ZIPCODE',
 'SHIPPING_CITY',
 'SHIPPING_STATE',
 'SHIPPING_ZIPCODE',
 'EMAIL',
 'IP_ADDRESS',
 'SALES_CHANNEL',
 'PHONE_MATCHES',
 'NUM_BYOD',
 'BYOD_INDICATOR',
 'FIRST_PARTY_ID_SCORE',
 'THIRD_PARTY_ID_SCORE',
 'ORDER_TYPE',
 'BLACKLIST_EMAIL',
 'FPF_HIGH_YELLOW',
 'TPF_HIGH_YELLOW',
 'SSN_BLACKLISTED',
 'DEVICE_AT_HOME',
 'ORDER_EXCHANGE',
 'YELLOW_SKU',
 'EXPERIAN_R

In [141]:
order_extract_df['ACTIVE_WATCH_RESULT'].head()     # use this one for prediction target

0      RED
1    GREEN
2    GREEN
3    GREEN
4    GREEN
Name: ACTIVE_WATCH_RESULT, dtype: object

(247838,)

In [146]:
order_extract_df['ACTIVE_WATCH_RESULT'].unique()

array(['RED', 'GREEN', 'YELLOW', nan, 'BLACK'], dtype=object)

In [147]:
order_extract_df[order_extract_df['ACTIVE_WATCH_RESULT'].isna()].shape

(3684, 103)

In [148]:
order_extract_df.shape

(247838, 103)

In [149]:
order_extract_df.dropna(subset=['ACTIVE_WATCH_RESULT'], inplace=True)

In [275]:
order_extract_df.shape

(244154, 103)

In [288]:
# n = 'ACTIVE_WATCH_RESULT'
# categories = {}
# order_extract_df.loc[:,n] = order_extract_df.loc[:,n].astype('category').cat.as_ordered()
# categories[n] = order_extract_df[n].cat.categories
# order_extract_df.loc[:,n] = pd.Categorical(order_extract_df[n], categories=categories[n], ordered=True)

In [301]:
target_df = order_extract_df[['ACTIVE_WATCH_RESULT']]

In [302]:
target_df.columns

Index(['ACTIVE_WATCH_RESULT'], dtype='object')

In [303]:
def convert_label(label_string):
    if label_string == 'RED':
        return 0
    if label_string == 'GREEN':
        return 1
    if label_string == 'YELLOW':
        return 2
    return 3       # BLACK
    

In [304]:
target_df['ACTIVE_WATCH_RESULT'] = target_df['ACTIVE_WATCH_RESULT'].apply(lambda x: convert_label(x))

/anaconda/envs/azureml_py36/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [306]:
target_df['ACTIVE_WATCH_RESULT'].unique()

[0, 1, 2, 3]
Categories (4, int64): [3 < 1 < 0 < 2]

In [307]:
target_df.shape

(244154, 1)

In [308]:
target_df.to_csv('ivr_backup_data/target_df.csv', header=True, index=False)

In [152]:
# order_extract_df['FRAUDSTATUSIND'].head()

In [153]:
order_extract_df.head()

,ORDER_ID,ORDER_CREATED_DATE,ACCOUNT_NUMBER,COMCAST_ACCOUNT_NUMBER,FRAUDNET_SCORE,TOTAL_PRICE,SKU,TYPE1,MAKE1,MODEL1,...,XM_ACCOUNT_OPENDATE,WRITEOFFIND,COLLECTIONSTATUS,ACCOUNT_OPEN_TO_ORDER_MINS,START_TO_CLOSE_MINS,START_TO_CASECLOSE_MINS,CASECLOSE_TO_ORDERCLOSE_MINS,CASEOPEN_TO_CASECLOSE_MINS,ORDER_RISK_LABEL,ORDER_RISK_REASON
0,187983116,2019-05-25 12:52:01,716308354,3364280765,NaN,1554.60,"[""723755019478"",""723755019478"",""190198451835"",...",Phone,Apple,iPhone 8,...,2018-12-08 10:26:59,R,NaN,241525.00,NaN,NaN,NaN,NaN,1,Did not clear Fraud Review
1,225253548,2019-10-19 19:32:01,2387138088,2735549480,NaN,0.00,"[""APLIPHONE6GOLD64""]",Phone,Apple,iPhone 6,...,2019-06-10 05:57:06,R,Not in Collection,188914.90,8988.82,NaN,NaN,NaN,0,Good Order
2,215951933,2019-09-13 15:18:21,2297289602,3659356712,NaN,1080.09,"[""190199392588""]",Phone,Apple,iPhone 11 Pro,...,2019-09-13 03:02:21,R,Not in Collection,195.95,14938.45,NaN,NaN,NaN,0,Good Order
3,217300035,2019-09-18 21:36:05,2747556495,2019265105,NaN,593.97,"[""190198453655""]",Phone,Apple,iPhone 8 Plus,...,2017-11-18 08:23:19,N,Not in Collection,963612.72,4198.95,NaN,NaN,NaN,0,Good Order
4,222110639,2019-10-07 02:25:39,3159523017,4140934545,NaN,0.00,"[""190198155795""]",Phone,Apple,iPhone 7 Plus,...,2019-09-15 11:00:12,R,Not in Collection,30625.43,15954.00,NaN,NaN,NaN,0,Good Order


In [154]:
order_extract_df.columns

Index(['ORDER_ID', 'ORDER_CREATED_DATE', 'ACCOUNT_NUMBER',
       'COMCAST_ACCOUNT_NUMBER', 'FRAUDNET_SCORE', 'TOTAL_PRICE', 'SKU',
       'TYPE1', 'MAKE1', 'MODEL1',
       ...
       'XM_ACCOUNT_OPENDATE', 'WRITEOFFIND', 'COLLECTIONSTATUS',
       'ACCOUNT_OPEN_TO_ORDER_MINS', 'START_TO_CLOSE_MINS',
       'START_TO_CASECLOSE_MINS', 'CASECLOSE_TO_ORDERCLOSE_MINS',
       'CASEOPEN_TO_CASECLOSE_MINS', 'ORDER_RISK_LABEL', 'ORDER_RISK_REASON'],
      dtype='object', length=103)

The fields that are available during order entry are marked in the Risk_Model.csv. Extracted earlier.

In [155]:
available_fields = [line.rstrip('\n') for line in open('ivr_data/available_fields.txt')]

In [156]:
available_fields.remove('ALL_SKUS')
available_fields.append('SKU')

In [157]:
available_fields

['MBC_ORDER_ID',
 'ORDER_CREATED_DATE',
 'ACCOUNT_NUMBER',
 'STATUS',
 'FRAUDNET_SCORE',
 'TOTAL_PRICE',
 'MODEL1',
 'TYPE1',
 'MAKE1',
 'PRICE1',
 'MODEL2',
 'TYPE2',
 'MAKE2',
 'PRICE2',
 'MODEL3',
 'TYPE3',
 'MAKE3',
 'PRICE3',
 'MODEL4',
 'TYPE4',
 'MAKE4',
 'PRICE4',
 'MODEL5',
 'TYPE5',
 'MAKE5',
 'PRICE5',
 'PORT_IN_FLAG',
 'NUM_PORTIN',
 'FROM_CARRIER_NAME',
 'ACNT_BILL_FNAME_MATCHES',
 'ACNT_BILL_LNAME_MATCHES',
 'ACNT_SHIP_FNAME_MATCHES',
 'ACNT_SHIP_LNAME_MATCHES',
 'SHIP_BILL_FNAME_MATCHES',
 'SHIP_BILL_LNAME_MATCHES',
 'BILL_SHIP_ADDR_MATCHES',
 'EMAIL',
 'BILLING_CITY',
 'BILLING_STATE',
 'BILLING_ZIPCODE',
 'SHIPPING_CITY',
 'SHIPPING_STATE',
 'SHIPPING_ZIPCODE',
 'SERVICE_CITY',
 'SERVICE_STATE',
 'SERVICE_ZIPCODE',
 'IP_ADDRESS',
 'SALES_CHANNEL',
 'FULFILLMENT_METHOD',
 'PHONE_MATCHES',
 'ORDER_PHONE_NUMBER',
 'NUM_BYOD',
 'BYOD_INDICATOR',
 'IS_EXISTING_CUSTOMER',
 'CART_TYPE',
 'INSTALLMENT_AMOUNT',
 'ONETIMECHARGE',
 'MONTHLYRECURRINGCHARGE',
 'SHIPPINGMETHOD',
 'L

Remove fields that Bharath did not extract and adjust column name as needed.

In [158]:
available_fields.remove('MBC_ORDER_ID')
available_fields.remove('STATUS')
# available_fields.remove('ALL_SKUS')
available_fields.remove('SERVICE_CITY')
available_fields.remove('SERVICE_STATE')
available_fields.remove('SERVICE_ZIPCODE')
available_fields.remove('FULFILLMENT_METHOD')
available_fields.remove('ORDER_PHONE_NUMBER')
available_fields.remove('CART_TYPE')
available_fields.remove('SSN_MISMATCH')
available_fields.remove('ZIP_MISMATCH')
available_fields.remove('ACCOUNT_OPENDATE')

# 'MBC_ORDER_ID' 'STATUS' 'ALL_SKUS' 'SERVICE_CITY' 'SERVICE_STATE'\n 'SERVICE_ZIPCODE' 'FULFILLMENT_METHOD' 'ORDER_PHONE_NUMBER' 'CART_TYPE'\n 'SSN_MISMATCH' 'ZIP_MISMATCH' 'ACCOUNT_OPENDATE'

In [159]:
good_order_df = order_extract_df[available_fields]

In [160]:
good_order_df.shape

(244154, 74)

In [161]:
good_order_df['ORDER_CREATED_DATE'].head()

0    2019-05-25 12:52:01
1    2019-10-19 19:32:01
2    2019-09-13 15:18:21
3    2019-09-18 21:36:05
4    2019-10-07 02:25:39
Name: ORDER_CREATED_DATE, dtype: object

In [162]:
good_order_df.columns

Index(['ORDER_CREATED_DATE', 'ACCOUNT_NUMBER', 'FRAUDNET_SCORE', 'TOTAL_PRICE',
       'MODEL1', 'TYPE1', 'MAKE1', 'PRICE1', 'MODEL2', 'TYPE2', 'MAKE2',
       'PRICE2', 'MODEL3', 'TYPE3', 'MAKE3', 'PRICE3', 'MODEL4', 'TYPE4',
       'MAKE4', 'PRICE4', 'MODEL5', 'TYPE5', 'MAKE5', 'PRICE5', 'PORT_IN_FLAG',
       'NUM_PORTIN', 'FROM_CARRIER_NAME', 'ACNT_BILL_FNAME_MATCHES',
       'ACNT_BILL_LNAME_MATCHES', 'ACNT_SHIP_FNAME_MATCHES',
       'ACNT_SHIP_LNAME_MATCHES', 'SHIP_BILL_FNAME_MATCHES',
       'SHIP_BILL_LNAME_MATCHES', 'BILL_SHIP_ADDR_MATCHES', 'EMAIL',
       'BILLING_CITY', 'BILLING_STATE', 'BILLING_ZIPCODE', 'SHIPPING_CITY',
       'SHIPPING_STATE', 'SHIPPING_ZIPCODE', 'IP_ADDRESS', 'SALES_CHANNEL',
       'PHONE_MATCHES', 'NUM_BYOD', 'BYOD_INDICATOR', 'IS_EXISTING_CUSTOMER',
       'INSTALLMENT_AMOUNT', 'ONETIMECHARGE', 'MONTHLYRECURRINGCHARGE',
       'SHIPPINGMETHOD', 'LINE_LIMIT_REACHED', 'DENIAL_OF_SERVICES',
       'BYOD_RETRY_COUNT', 'PREORDER_LINES', 'ALLOWED_LINES',


In [163]:
# good_order_df[good_order_df['ACCOUNT_NUMBER']==2173973859]

There are 40,934 zipcodes in US. Drop it for now

In [164]:
good_order_df.drop(['BILLING_ZIPCODE'], axis=1, inplace=True)
good_order_df.drop(['SHIPPING_ZIPCODE'], axis=1, inplace=True)

Also drop the cities. Too sparse to incorporate

In [165]:
good_order_df.drop(['SHIPPING_CITY'], axis=1, inplace=True)
good_order_df.drop(['BILLING_CITY'], axis=1, inplace=True)

Check the colunmns with nan. There all need to be handled

In [166]:
NaN_list = good_order_df.columns[good_order_df.isna().any()].tolist()

In [167]:
len(NaN_list)

30

In [168]:
NaN_list

['FRAUDNET_SCORE',
 'PRICE1',
 'MODEL2',
 'TYPE2',
 'MAKE2',
 'PRICE2',
 'MODEL3',
 'TYPE3',
 'MAKE3',
 'PRICE3',
 'MODEL4',
 'TYPE4',
 'MAKE4',
 'PRICE4',
 'MODEL5',
 'TYPE5',
 'MAKE5',
 'PRICE5',
 'PORT_IN_FLAG',
 'FROM_CARRIER_NAME',
 'BILLING_STATE',
 'SHIPPING_STATE',
 'SHIPPINGMETHOD',
 'PREORDER_LINES',
 'INTERNAL_CREDIT_CHECK_DONE',
 'ORDER_EXCHANGE',
 'EXPERIAN_RESPONSE_1',
 'EXPERIAN_RESPONSE_2',
 'IDA_RESULT',
 'FRAUDNET_RESULT']

In [169]:
# good_order_df.dtypes

Handle numerical and categorical variables separately

In [170]:
order_num_df = good_order_df.select_dtypes(include=np.number)

In [171]:
num_columns = order_num_df.columns.tolist()

There were string values captured in PRICE1 and PRICE2, so they didn't get picked up automatically. have to manually incorported them

In [172]:
num_columns.append('PRICE1')
num_columns.append('PRICE2')
# ORDER_CREATED_DATE is a numerical field after conversion to unix time
num_columns.append('ORDER_CREATED_DATE')

num_columns.remove('ACCOUNT_NUMBER')     # ACCOUNT_NUMBER should not be treated as a number


In [173]:
num_columns

['FRAUDNET_SCORE',
 'TOTAL_PRICE',
 'PRICE3',
 'PRICE4',
 'PRICE5',
 'NUM_PORTIN',
 'ACNT_BILL_FNAME_MATCHES',
 'ACNT_BILL_LNAME_MATCHES',
 'ACNT_SHIP_FNAME_MATCHES',
 'ACNT_SHIP_LNAME_MATCHES',
 'SHIP_BILL_FNAME_MATCHES',
 'SHIP_BILL_LNAME_MATCHES',
 'BILL_SHIP_ADDR_MATCHES',
 'EMAIL',
 'IP_ADDRESS',
 'PHONE_MATCHES',
 'NUM_BYOD',
 'INSTALLMENT_AMOUNT',
 'ONETIMECHARGE',
 'MONTHLYRECURRINGCHARGE',
 'BYOD_RETRY_COUNT',
 'PREORDER_LINES',
 'ALLOWED_LINES',
 'APPROVED_LINES',
 'EXTERNAL_APPROVED_LINES',
 'FIRST_PARTY_ID_SCORE',
 'THIRD_PARTY_ID_SCORE',
 'PRICE1',
 'PRICE2',
 'ORDER_CREATED_DATE']

Create a separate DataFrame to process the numerical columns

In [174]:
order_num_df = good_order_df[num_columns]

Which of the numerical columns contain nan?

In [175]:
order_num_df.columns[order_num_df.isna().any()].tolist()

['FRAUDNET_SCORE',
 'PRICE3',
 'PRICE4',
 'PRICE5',
 'PREORDER_LINES',
 'PRICE1',
 'PRICE2']

Fill nan values with 0

In [176]:
order_num_df['PREORDER_LINES'].fillna(0, inplace=True)
order_num_df['FRAUDNET_SCORE'].fillna(0, inplace=True)
# order_num_df['PRICE1'].fillna(0, inplace=True)
# order_num_df['PRICE2'].fillna(0, inplace=True)
order_num_df['PRICE3'].fillna(0, inplace=True)
order_num_df['PRICE4'].fillna(0, inplace=True)
order_num_df['PRICE5'].fillna(0, inplace=True)

Double check 0's got filled in

In [177]:
order_num_df[['FRAUDNET_SCORE', 'PRICE3','PRICE4','PRICE5', 'PREORDER_LINES']].head()

,FRAUDNET_SCORE,PRICE3,PRICE4,PRICE5,PREORDER_LINES
0,0.0,119.99,119.99,0.0,0.0
1,0.0,0.00,0.00,0.0,0.0
2,0.0,0.00,0.00,0.0,0.0
3,0.0,0.00,0.00,0.0,0.0
4,0.0,0.00,0.00,0.0,0.0


Convert PRICE1 and PRICES2 to numerical values

In [178]:
def convert_to_float(price_string):
    try:
        return float(price_string)
    except:
        print('price_string={}'.format(price_string))
        return 0

In [179]:
order_num_df[['PRICE1', 'PRICE2']].head()

,PRICE1,PRICE2
0,599.99,599.99
1,0.00,NaN
2,999.99,NaN
3,549.99,NaN
4,469.99,NaN


In [180]:
order_num_df['PRICE1'] = order_num_df['PRICE1'].apply(lambda x: convert_to_float(x))

price_string=153 NGP
price_string=153 NGP
price_string=153 NGP
price_string=153 NGP
price_string=153 NGP
price_string=153 NGP
price_string=153 NGP
price_string=153 NGP
price_string=153 NGP
price_string=153 NGP
price_string=153 NGP
price_string=153 NGP
price_string=153 NGP
price_string=153 NGP
price_string=153 NGP
price_string=153 NGP
price_string=153 NGP
price_string=153 NGP
price_string=153 NGP
price_string=153 NGP
price_string=153 NGP


/anaconda/envs/azureml_py36/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [181]:
order_num_df['PRICE2'] = order_num_df['PRICE2'].apply(lambda x: convert_to_float(x))

price_string=153 NGP


/anaconda/envs/azureml_py36/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [182]:
# order_num_df['PRICE1'].isna()
order_num_df['PRICE1'].fillna(0, inplace=True)
order_num_df['PRICE2'].fillna(0, inplace=True)

PRICE1 and PRICE2 should now be all numerical, and have 0's where they used to be nan

In [183]:
order_num_df[['PRICE1', 'PRICE2']].head()

,PRICE1,PRICE2
0,599.99,599.99
1,0.00,0.00
2,999.99,0.00
3,549.99,0.00
4,469.99,0.00


In [184]:
# order_num_df.iloc[100]['ORDER_CREATED_DATE']

In [185]:
import time
import datetime

def convert_to_unix_time(date_time_string):
    return time.mktime(datetime.datetime.strptime(date_time_string, "%Y-%m-%d %H:%M:%S").timetuple())


In [186]:
order_num_df['ORDER_CREATED_DATE'] = order_num_df['ORDER_CREATED_DATE'].apply(lambda x: convert_to_unix_time(x))

# order_num_df['ORDER_CREATED_DATE_vector'].head()

# order_num_df.drop(['ORDER_CREATED_DATE'], axis=1, inplace=True)

/anaconda/envs/azureml_py36/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [187]:
order_num_df.columns[order_num_df.isna().any()].tolist()

[]

In [188]:
order_num_df.shape

(244154, 30)

In [189]:
order_num_df.to_csv('ivr_backup_data/order_num_df.csv', index=False)

Get all the numerical column names

In [190]:
num_columns

['FRAUDNET_SCORE',
 'TOTAL_PRICE',
 'PRICE3',
 'PRICE4',
 'PRICE5',
 'NUM_PORTIN',
 'ACNT_BILL_FNAME_MATCHES',
 'ACNT_BILL_LNAME_MATCHES',
 'ACNT_SHIP_FNAME_MATCHES',
 'ACNT_SHIP_LNAME_MATCHES',
 'SHIP_BILL_FNAME_MATCHES',
 'SHIP_BILL_LNAME_MATCHES',
 'BILL_SHIP_ADDR_MATCHES',
 'EMAIL',
 'IP_ADDRESS',
 'PHONE_MATCHES',
 'NUM_BYOD',
 'INSTALLMENT_AMOUNT',
 'ONETIMECHARGE',
 'MONTHLYRECURRINGCHARGE',
 'BYOD_RETRY_COUNT',
 'PREORDER_LINES',
 'ALLOWED_LINES',
 'APPROVED_LINES',
 'EXTERNAL_APPROVED_LINES',
 'FIRST_PARTY_ID_SCORE',
 'THIRD_PARTY_ID_SCORE',
 'PRICE1',
 'PRICE2',
 'ORDER_CREATED_DATE']

In [191]:
len(num_columns)

30

One way to get all the categorical columns is use the statement below

In [192]:
good_order_df.select_dtypes(['object', 'category']).columns

Index(['ORDER_CREATED_DATE', 'MODEL1', 'TYPE1', 'MAKE1', 'PRICE1', 'MODEL2',
       'TYPE2', 'MAKE2', 'PRICE2', 'MODEL3', 'TYPE3', 'MAKE3', 'MODEL4',
       'TYPE4', 'MAKE4', 'MODEL5', 'TYPE5', 'MAKE5', 'PORT_IN_FLAG',
       'FROM_CARRIER_NAME', 'BILLING_STATE', 'SHIPPING_STATE', 'SALES_CHANNEL',
       'BYOD_INDICATOR', 'IS_EXISTING_CUSTOMER', 'SHIPPINGMETHOD',
       'LINE_LIMIT_REACHED', 'DENIAL_OF_SERVICES',
       'INTERNAL_CREDIT_CHECK_DONE', 'EXTERNAL_CREDIT_CHECK_DONE',
       'BLACKLIST_EMAIL', 'FPF_HIGH_YELLOW', 'TPF_HIGH_YELLOW',
       'SSN_BLACKLISTED', 'DEVICE_AT_HOME', 'ORDER_EXCHANGE', 'YELLOW_SKU',
       'EXPERIAN_RESPONSE_1', 'EXPERIAN_RESPONSE_2', 'IDA_RESULT',
       'FRAUDNET_RESULT', 'SKU'],
      dtype='object')

But let me instead get all columns and subtract the numerical ones. That way I am sure no columns got skipped

In [193]:
all_columns = good_order_df.columns.tolist()

In [194]:
all_columns

['ORDER_CREATED_DATE',
 'ACCOUNT_NUMBER',
 'FRAUDNET_SCORE',
 'TOTAL_PRICE',
 'MODEL1',
 'TYPE1',
 'MAKE1',
 'PRICE1',
 'MODEL2',
 'TYPE2',
 'MAKE2',
 'PRICE2',
 'MODEL3',
 'TYPE3',
 'MAKE3',
 'PRICE3',
 'MODEL4',
 'TYPE4',
 'MAKE4',
 'PRICE4',
 'MODEL5',
 'TYPE5',
 'MAKE5',
 'PRICE5',
 'PORT_IN_FLAG',
 'NUM_PORTIN',
 'FROM_CARRIER_NAME',
 'ACNT_BILL_FNAME_MATCHES',
 'ACNT_BILL_LNAME_MATCHES',
 'ACNT_SHIP_FNAME_MATCHES',
 'ACNT_SHIP_LNAME_MATCHES',
 'SHIP_BILL_FNAME_MATCHES',
 'SHIP_BILL_LNAME_MATCHES',
 'BILL_SHIP_ADDR_MATCHES',
 'EMAIL',
 'BILLING_STATE',
 'SHIPPING_STATE',
 'IP_ADDRESS',
 'SALES_CHANNEL',
 'PHONE_MATCHES',
 'NUM_BYOD',
 'BYOD_INDICATOR',
 'IS_EXISTING_CUSTOMER',
 'INSTALLMENT_AMOUNT',
 'ONETIMECHARGE',
 'MONTHLYRECURRINGCHARGE',
 'SHIPPINGMETHOD',
 'LINE_LIMIT_REACHED',
 'DENIAL_OF_SERVICES',
 'BYOD_RETRY_COUNT',
 'PREORDER_LINES',
 'ALLOWED_LINES',
 'INTERNAL_CREDIT_CHECK_DONE',
 'EXTERNAL_CREDIT_CHECK_DONE',
 'APPROVED_LINES',
 'EXTERNAL_APPROVED_LINES',
 'FIRST_P

In [195]:
# one field in order_num_columns is not in all_columns
# for num_column in num_columns:
#     if num_column not in all_columns:
#         print('{} not in all_columns'.format(num_column))

In [196]:
len(all_columns)

70

In [197]:
object_columns = list(set(all_columns) - set(num_columns))

In [198]:
object_columns

['EXPERIAN_RESPONSE_1',
 'EXPERIAN_RESPONSE_2',
 'ORDER_EXCHANGE',
 'MAKE1',
 'MAKE2',
 'TYPE5',
 'TPF_HIGH_YELLOW',
 'MODEL4',
 'IDA_RESULT',
 'BLACKLIST_EMAIL',
 'FRAUDNET_RESULT',
 'YELLOW_SKU',
 'SHIPPING_STATE',
 'FROM_CARRIER_NAME',
 'SALES_CHANNEL',
 'PORT_IN_FLAG',
 'SKU',
 'BILLING_STATE',
 'MAKE3',
 'BYOD_INDICATOR',
 'MAKE5',
 'MAKE4',
 'ACCOUNT_NUMBER',
 'MODEL3',
 'IS_EXISTING_CUSTOMER',
 'TYPE2',
 'TYPE3',
 'TYPE1',
 'LINE_LIMIT_REACHED',
 'TYPE4',
 'SSN_BLACKLISTED',
 'INTERNAL_CREDIT_CHECK_DONE',
 'MODEL2',
 'MODEL1',
 'MODEL5',
 'DEVICE_AT_HOME',
 'FPF_HIGH_YELLOW',
 'DENIAL_OF_SERVICES',
 'SHIPPINGMETHOD',
 'EXTERNAL_CREDIT_CHECK_DONE']

In [199]:
len(object_columns)

40

Next, create a DataFrame for all the categorical columns only

In [200]:
order_category_df = good_order_df[object_columns]

In [201]:
del good_order_df

In [202]:
# order_category_df['IS_EXISTING_CUSTOMER'].unique()

The SKU field is one of the complicated ones, because it is a list of devices. Each devices can come from a long list of available products

In [203]:
sku_list = order_category_df['SKU'].unique().tolist()

In [204]:
sku_list

['["723755019478","723755019478","190198451835","190198451781"]',
 '["APLIPHONE6GOLD64"]',
 '["190199392588"]',
 '["190198453655"]',
 '["190198155795"]',
 '["190199220577","190199380677"]',
 '["190198066497","190198066534"]',
 '["190198456700"]',
 '["652810830621"]',
 '["190199392564"]',
 '["190198456656"]',
 '["APLIPH8PSIL64N1X"]',
 '["APLIPH6PLUSSIL16"]',
 '["190199392526"]',
 '["887276363332"]',
 '["190198354426"]',
 '["887276348490"]',
 '["190199380714"]',
 '["190198786029","191058084972"]',
 '["190198776440","848467076929"]',
 '["190199220652"]',
 '["887276246512"]',
 '["APLIPHONE6SILV16"]',
 '["888462734868"]',
 '["190199220553"]',
 '["190199220539"]',
 '["190199220652","190199220737"]',
 '["190199220751"]',
 '["190198776426"]',
 '["190199380738"]',
 '["APLIPHONE6GRAY64","SEASMG950UBLCK64"]',
 '["190198451835"]',
 '["190198043375"]',
 '["APLIPHONE6GRAY32"]',
 '["190199392663"]',
 '["190198058614"]',
 '["887276348490","887276348490"]',
 '["APLIPH8GOLD64N1X"]',
 '["888462501958","A

Take an inventory of all devices. Hopefully, this is full list. Otherwise, trouble down the road.

In [205]:

def split_sku(sku_string):
#     print('sku_string={}'.format(sku_string))
    skus = [np.nan] * 9
#     sku_2 = np.nan
#     sku_3 = np.nan
#     sku_4 = np.nan
#     sku_5 = np.nan
    truncated_string = sku_string[1:-1]
#     print('truncated_string={}'.format(truncated_string))
    sku_array = truncated_string.split(',')
    no_skus = len(sku_array)
    i = 0
    for sku in sku_array:
#         print('skus len={}'.format(len(skus)))
#         print('i = {}'.format(i))
        sku = sku.replace("\"", "")
        try:
#             print('sku={}'.format(sku))
#             skus[i] = sku
            all_skus.add(sku)
        except Exception as e:
            print('sku_string={}'.format(sku_string))
            print("I/O error({})".format(e))
        i += 1
#         if i > 3:
#             break
        
#     return pd.Series([skus[0], skus[1], skus[2], skus[3], skus[4]],index=['SKU_1','SKU_2','SKU_3','SKU_4','SKU_5'])


In [206]:
all_skus = set()

for row in order_category_df.iterrows():
#     print(row[1])
#     print(row[1]['SKU'])
    split_sku(row[1]['SKU'])

In [207]:
len(all_skus)

609

Create a map that gives each device an unique index

In [208]:
# create dictionary to map index for the devices
device_map = {}
index = 1          # reserve 0 for nan entries
for device in all_skus:
    device_map[device] = index
    index += 1

In [209]:
device_count = len(device_map)
device_count

609

In [210]:
device_map['846127180283']       # check a sample

140

This encoding method assigns the value 100 to each device ordered, and add a value 10 if it is BYOD
The exact value is arbitrary and will not have any bearing on the final embedded value

In [211]:
ACTIVE = 100
BYOD = 10

# def generate_device_vector(row):
def generate_device_vector(sku_string, byod_string):
#     print('row = {}'.format(row))
#     print('sku_string={}'.format(sku_string))
#     print('byod_string={}'.format(byod_string))
    SKU_vector = [0] * (device_count + 1)

    truncated_string = sku_string[1:-1]
# #     print('truncated_string={}'.format(truncated_string))
    sku_array = truncated_string.split(',')
    truncated_byod_string = byod_string[1:-1]
    byod_array = truncated_byod_string.split(',')
#     no_skus = len(sku_array)
    i = 0
    for sku in sku_array:
# #         print('skus len={}'.format(len(skus)))
# #         print('i = {}'.format(i))
        sku = sku.replace("\"", "")
        byod = byod_array[i].replace("\"", "").replace("''", "")
        try:
#             print('sku={}'.format(sku))
#             print('byod={}'.format(byod))
            device_index = device_map[sku]
            num_rep = 100         
            if byod.lower() == 'y':
#                 print('found byod')
                num_rep += 10     # if byod, add 10
#             print('num_rep={}'.format(num_rep))
            SKU_vector[device_index] = num_rep
        except Exception as e:
            print('sku_string={}'.format(sku_string))
            print("I/O error({})".format(e))
        i += 1
#         if i > 3:
#             break
        
    return SKU_vector
    

In [212]:
# create the vector for devices
# I will use dimension 20
# active devices is 100
# byod is 10
order_category_df['SKU_vector'] = order_category_df.apply(lambda x: generate_device_vector(x['SKU'], x['BYOD_INDICATOR']), axis=1)

In [213]:
device_map

{'190199220652': 1,
 '190199380837': 2,
 'APLIP8PGLD256N1X': 3,
 'SEASMG960UPUR128': 4,
 'APLIP8PRED256N1X': 5,
 '190198776501': 6,
 '660543513506': 7,
 '887276284392': 8,
 'APLIPH6SPLROSE64': 9,
 '660543460480': 10,
 'APLIPH6SPLGOLD16': 11,
 '190198360083': 12,
 '887276206837': 13,
 '190198786128': 14,
 '190199285446': 15,
 '190199220775': 16,
 '660543507444': 17,
 '190198066497': 18,
 '190199392540': 19,
 '660543494829': 20,
 '190198790439': 21,
 '660543472551': 22,
 'APLIPH6PLGOLD128': 23,
 '660543511373': 24,
 '815373020841': 25,
 '840056102972': 26,
 '660543469322': 27,
 'SEASMG950UBLUE64': 28,
 '848467076530': 29,
 'SEASMG975UBLK128': 30,
 '190198857842': 31,
 '848709058065': 32,
 'SEASMG960UGLD128': 33,
 '190198066558': 34,
 '190198776402': 35,
 '190199220591': 36,
 '190198786067': 37,
 '190198043399': 38,
 '887276322353': 39,
 '190198790637': 40,
 '842776114952': 41,
 '190199291782': 42,
 'APLIPH6PLUSGLD16': 43,
 '190199380813': 44,
 'SEASMG973UWHT128': 45,
 '660543512585': 46,

In [214]:
# order_category_df.iloc[100]['SKU_vector']

We don't need the original columns SKU and BYOD_INDICATOR any more. Their values are now represented by SKU_vector

In [215]:
order_category_df.drop(['SKU', 'BYOD_INDICATOR'], axis=1, inplace=True)

In [216]:
order_category_df.columns

Index(['EXPERIAN_RESPONSE_1', 'EXPERIAN_RESPONSE_2', 'ORDER_EXCHANGE', 'MAKE1',
       'MAKE2', 'TYPE5', 'TPF_HIGH_YELLOW', 'MODEL4', 'IDA_RESULT',
       'BLACKLIST_EMAIL', 'FRAUDNET_RESULT', 'YELLOW_SKU', 'SHIPPING_STATE',
       'FROM_CARRIER_NAME', 'SALES_CHANNEL', 'PORT_IN_FLAG', 'BILLING_STATE',
       'MAKE3', 'MAKE5', 'MAKE4', 'ACCOUNT_NUMBER', 'MODEL3',
       'IS_EXISTING_CUSTOMER', 'TYPE2', 'TYPE3', 'TYPE1', 'LINE_LIMIT_REACHED',
       'TYPE4', 'SSN_BLACKLISTED', 'INTERNAL_CREDIT_CHECK_DONE', 'MODEL2',
       'MODEL1', 'MODEL5', 'DEVICE_AT_HOME', 'FPF_HIGH_YELLOW',
       'DENIAL_OF_SERVICES', 'SHIPPINGMETHOD', 'EXTERNAL_CREDIT_CHECK_DONE',
       'SKU_vector'],
      dtype='object')

In [217]:
# order_category_df.head()

Take a peek how many of the categorical values contain nan

In [218]:
order_category_df.columns[order_category_df.isna().any()].tolist()

['EXPERIAN_RESPONSE_1',
 'EXPERIAN_RESPONSE_2',
 'ORDER_EXCHANGE',
 'MAKE2',
 'TYPE5',
 'MODEL4',
 'IDA_RESULT',
 'FRAUDNET_RESULT',
 'SHIPPING_STATE',
 'FROM_CARRIER_NAME',
 'PORT_IN_FLAG',
 'BILLING_STATE',
 'MAKE3',
 'MAKE5',
 'MAKE4',
 'MODEL3',
 'TYPE2',
 'TYPE3',
 'TYPE4',
 'INTERNAL_CREDIT_CHECK_DONE',
 'MODEL2',
 'MODEL5',
 'SHIPPINGMETHOD']

PORT_IN_FLAG and FROM_CARRIER_NAME are two other fields that should be combined

In [219]:
order_category_df[['PORT_IN_FLAG', 'FROM_CARRIER_NAME']].head()

,PORT_IN_FLAG,FROM_CARRIER_NAME
0,"[""N"",""N"",""N"",""N""]","[null,null,null,null]"
1,"[""Y""]","[""MetroPCS""]"
2,"[""Y""]","[""GoogleVoice""]"
3,"[""N""]",[null]
4,"[""Y""]","[""Other/NotSure""]"


In [220]:
def isNaN(num):
    return num != num

In [221]:
# this function takes the string value, and generate a python list out of it
def split_carriers(carrier_string):
    if isNaN(carrier_string):
        return
#     print('carrier_string={}'.format(carrier_string))
    carriers = [np.nan] * 5

    truncated_string = carrier_string[1:-1]
#     print('truncated_string={}'.format(truncated_string))
    carrier_array = truncated_string.split(',')
    no_carriers = len(carrier_array)
    i = 0
    for carrier in carrier_array:
#         print('carrier_array len={}'.format(len(carrier_array)))
#         print('i = {}'.format(i))
        carrier = carrier.replace("\"", "")
        try:
#             print('carrier={}'.format(carrier))
#             carriers[i] = carrier
            all_carriers.add(carrier)
        except Exception as e:
            print('carrier_string={}'.format(carrier_string))
            print("error({})".format(e))
        i += 1
#         if i > 3:
#             break
        

Take stock of what carriers are there

In [222]:
all_carriers = set()

for row in order_category_df.iterrows():
#     print(row[1])
#     print(row[1]['SKU'])
    split_carriers(row[1]['FROM_CARRIER_NAME'])

In [223]:
all_carrier_list = list(all_carriers)

In [224]:
len(all_carrier_list)

22

In [225]:
all_carrier_list

['TelcelAmerica',
 'GoogleVoice',
 'Sprint',
 'VerizonPrepaid',
 'StraightTalk',
 'null',
 'T-Mobile',
 'TracFone',
 'GoogleFi',
 'U.S.Cellular',
 'Cricket',
 'Net10',
 'Boost Mobile',
 'Other/NotSure',
 'RepublicWireless',
 'AT&TWireless',
 'T-MobilePrepaid',
 'FreedomPop',
 'VirginMobile',
 'VerizonWireless',
 'AT&TPrepaid',
 'MetroPCS']

In [226]:
# move null to top of the list
all_carrier_list.remove('null')
all_carrier_list.insert(0, 'null')

In [227]:
all_carrier_list     # now null is the first one

['null',
 'TelcelAmerica',
 'GoogleVoice',
 'Sprint',
 'VerizonPrepaid',
 'StraightTalk',
 'T-Mobile',
 'TracFone',
 'GoogleFi',
 'U.S.Cellular',
 'Cricket',
 'Net10',
 'Boost Mobile',
 'Other/NotSure',
 'RepublicWireless',
 'AT&TWireless',
 'T-MobilePrepaid',
 'FreedomPop',
 'VirginMobile',
 'VerizonWireless',
 'AT&TPrepaid',
 'MetroPCS']

In [228]:
carrier_map = {}
index = 0          # already accounted for nan
for carrier in all_carrier_list:
    carrier_map[carrier] = index
    index += 1

In [229]:
carrier_map

{'null': 0,
 'TelcelAmerica': 1,
 'GoogleVoice': 2,
 'Sprint': 3,
 'VerizonPrepaid': 4,
 'StraightTalk': 5,
 'T-Mobile': 6,
 'TracFone': 7,
 'GoogleFi': 8,
 'U.S.Cellular': 9,
 'Cricket': 10,
 'Net10': 11,
 'Boost Mobile': 12,
 'Other/NotSure': 13,
 'RepublicWireless': 14,
 'AT&TWireless': 15,
 'T-MobilePrepaid': 16,
 'FreedomPop': 17,
 'VirginMobile': 18,
 'VerizonWireless': 19,
 'AT&TPrepaid': 20,
 'MetroPCS': 21}

The above are the discovered carrier

How many unique combination of carriers are there?

In [230]:
order_category_df['FROM_CARRIER_NAME'].unique()

array(['[null,null,null,null]', '["MetroPCS"]', '["GoogleVoice"]',
       '[null]', '["Other/NotSure"]',
       '["VerizonWireless","VerizonWireless"]', '["Boost Mobile",null]',
       '["VerizonWireless"]', '["Sprint"]', '["T-Mobile"]',
       '["AT&TWireless"]', '["GoogleFi"]', nan, '["AT&TPrepaid"]',
       '["AT&TWireless","AT&TWireless"]', '["VirginMobile"]',
       '[null,null]',
       '["VerizonWireless","VerizonWireless","VerizonWireless","VerizonWireless","VerizonWireless"]',
       '["VerizonWireless","VerizonWireless","VerizonWireless"]',
       '["Sprint","Sprint"]', '["TracFone"]',
       '["VerizonWireless","VerizonWireless","VerizonWireless","VerizonWireless"]',
       '["Sprint","Sprint","Sprint"]', '["StraightTalk"]',
       '["RepublicWireless","RepublicWireless","RepublicWireless",null]',
       '["StraightTalk","StraightTalk"]', '["AT&TWireless","T-Mobile"]',
       '["Cricket"]', '["Cricket","Cricket"]', '[null,null,null]',
       '["Other/NotSure",null]', '["T-Mo

In [231]:
def generate_carrier_vector(carrier_string):
#     print('row = {}'.format(row))

    carrier_vector = [0] * 5       # 5 carriers max. 0 means null

    try:
        truncated_string = carrier_string[1:-1]
    except:
#         print('carrier_string={}'.format(carrier_string))
        return carrier_vector
# #     print('truncated_string={}'.format(truncated_string))
    carrier_array = truncated_string.split(',')
    i = 0
    for carrier in carrier_array:
        carrier = carrier.replace("\"", "")

        try:
#             print('carrier={}'.format(carrier))
#             print('byod={}'.format(byod))
            carrier_index = carrier_map[carrier]
            carrier_vector[i] = carrier_index
        except Exception as e:
            print('sku_string={}'.format(carrier_string))
            print("I/O error({})".format(e))
        i += 1
        
    return carrier_vector

In [232]:
order_category_df['carrier_vector'] = order_category_df.apply(lambda x: generate_carrier_vector(x['FROM_CARRIER_NAME']), axis=1)

In [233]:
order_category_df.drop(['FROM_CARRIER_NAME'], axis=1, inplace=True)
order_category_df.drop(['PORT_IN_FLAG'], axis=1, inplace=True)

Now process Make, Type and Model

In [234]:
def collect_make(make_string1, make_string2, make_string3, make_string4, make_string5):
    all_makes.add(make_string1)
    all_makes.add(make_string2)
    all_makes.add(make_string3)
    all_makes.add(make_string4)
    all_makes.add(make_string5)

In [235]:
all_makes = set()
for row in order_category_df.iterrows():
#     print(row[1])
#     print(row[1]['SKU'])
    collect_make(row[1]['MAKE1'], row[1]['MAKE2'], row[1]['MAKE3'], row[1]['MAKE4'], row[1]['MAKE5'], )
    


In [236]:
all_make_list = list(all_makes)

In [237]:
all_make_list     # in a list, nan automatically floats to top

[nan,
 'Incipio',
 'LG',
 'Google',
 'SanDisk',
 'Samsung',
 'Kate Spade',
 'Case-Mate',
 'Motorola',
 'Mophie',
 'Tech21',
 'Gear4',
 'Ventev',
 'Superior',
 'Apple',
 'Speck',
 'PopSockets',
 'Sonix',
 'OtterBox',
 'Zagg']

In [238]:
len(all_make_list)

20

In [239]:
make_map = {}
index = 0          # already accounted for nan
for make in all_makes:
    make_map[make] = index
    index += 1

In [240]:
make_map

{nan: 0,
 'Incipio': 1,
 'LG': 2,
 'Google': 3,
 'SanDisk': 4,
 'Samsung': 5,
 'Kate Spade': 6,
 'Case-Mate': 7,
 'Motorola': 8,
 'Mophie': 9,
 'Tech21': 10,
 'Gear4': 11,
 'Ventev': 12,
 'Superior': 13,
 'Apple': 14,
 'Speck': 15,
 'PopSockets': 16,
 'Sonix': 17,
 'OtterBox': 18,
 'Zagg': 19}

A Generic function which creates a vector for the 5 fields of MAKE, TYPE, AND MODEL

In [241]:
def generate_column_vector(column_strings, column_map):
    len_map = len(column_map)
#     print('row = {}'.format(row))

    column_vector = [0] * len_map       # 5 carriers max. 0 means null

    for column in column_strings:

        try:
#             print('column={}'.format(column))
            column_index = column_map[column]
            column_vector[column_index] = column_vector[column_index] + 1    # for each occurrence of a column, add 1
        except Exception as e:
            print('column_string={}'.format(column_strings))
            print("I/O error({})".format(e))
        
    return column_vector

In [242]:
# def generate_make_vector(make_string1, make_string2, make_string3, make_string4, make_string5):
# #     print('row = {}'.format(row))

#     make_vector = [0] * 5       # 5 carriers max. 0 means nan

# # #     print('truncated_string={}'.format(truncated_string))
# #     no_skus = len(sku_array)
#     try:
#         make1 = make_string1.replace("\"", "")
#         make1_index = make_map[make1]
#         make_vector[0] = make1_index
#     except Exception as e:
#         print('sku_string={}'.format(make_string1))
#         print("I/O error({})".format(e))
    
#     try:
#         make2 = make_string2.replace("\"", "")
#         make2_index = make_map[make2]
#         make_vector[1] = make2_index
#     except Exception as e:
#         print('sku_string={}'.format(make_string2))
#         print("I/O error({})".format(e))
        
#     try:
#         make3 = make_string3.replace("\"", "")
#         make3_index = make_map[make3]
#         make_vector[2] = make3_index
#     except Exception as e:
#         print('sku_string={}'.format(make_string3))
#         print("I/O error({})".format(e))
        
#     try:
#         make4 = make_string4.replace("\"", "")
#         make4_index = make_map[make4]
#         make_vector[3] = make4_index
#     except Exception as e:
#         print('sku_string={}'.format(make_string4))
#         print("I/O error({})".format(e))
        
#     try:
#         make5 = make_string5.replace("\"", "")
#         make5_index = make_map[make5]
#         make_vector[4] = make5_index
#     except Exception as e:
#         print('sku_string={}'.format(make_string5))
#         print("I/O error({})".format(e))
        
#     return make_vector

In [243]:
order_category_df['make_vector'] = order_category_df.apply(lambda x: generate_column_vector([x['MAKE1'], x['MAKE2'], x['MAKE3'], x['MAKE4'], x['MAKE5']], make_map), axis=1)

In [244]:
order_category_df['make_vector'].head()

0    [1, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 2, ...
1    [4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, ...
2    [4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, ...
3    [4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, ...
4    [4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, ...
Name: make_vector, dtype: object

In [245]:
order_category_df.drop(columns=['MAKE1', 'MAKE2', 'MAKE3', 'MAKE4', 'MAKE5'], inplace=True)

In [246]:
def collect_type(type_string1, type_string2, type_string3, type_string4, type_string5):
#     print('type_string1={}'.format(type_string1))
    all_types.add(type_string1)
    all_types.add(type_string2)
    all_types.add(type_string3)
    all_types.add(type_string4)
    all_types.add(type_string5)

In [247]:
all_types = set()
for row in order_category_df.iterrows():
    collect_type(row[1]['TYPE1'], row[1]['TYPE2'], row[1]['TYPE3'], row[1]['TYPE4'], row[1]['TYPE5'], )
  

In [248]:
all_type_list = list(all_types)
all_type_list

[nan, 'Accessory', 'Phone', 'Tablet', 'Watch']

In [249]:
all_type_list.remove(np.nan)
all_type_list.insert(0, np.nan)
all_type_list

[nan, 'Accessory', 'Phone', 'Tablet', 'Watch']

In [250]:
len(all_type_list)

5

In [251]:
type_map = {}
index = 0          # already accounted for nan
for type in all_type_list:
    type_map[type] = index
    index += 1

In [252]:
type_map

{nan: 0, 'Accessory': 1, 'Phone': 2, 'Tablet': 3, 'Watch': 4}

In [253]:
# def generate_type_vector(type_string1, type_string12, type_string3, type_string4, type_string5):
# #     print('row = {}'.format(row))

#     type_vector = [0] * 5       # 5 types max. 0 means nan

# # #     print('truncated_string={}'.format(truncated_string))
# #     no_skus = len(sku_array)
#     try:
#         type1 = type_string1.replace("\"", "")
#         type1_index = type_map[type1]
#         type_vector[0] = type1_index
#     except Exception as e:
#         print('sku_string={}'.format(type_string1))
#         print("I/O error({})".format(e))
    
#     try:
#         type2 = type_string2.replace("\"", "")
#         type2_index = type_map[make2]
#         type_vector[1] = type2_index
#     except Exception as e:
#         print('sku_string={}'.format(type_string2))
#         print("I/O error({})".format(e))
        
#     try:
#         type3 = type_string3.replace("\"", "")
#         type3_index = type_map[type3]
#         type_vector[2] = type3_index
#     except Exception as e:
#         print('sku_string={}'.format(type_string3))
#         print("I/O error({})".format(e))
        
#     try:
#         type4 = type_string4.replace("\"", "")
#         type4_index = type_map[make4]
#         type_vector[3] = type4_index
#     except Exception as e:
#         print('sku_string={}'.format(type_string4))
#         print("I/O error({})".format(e))
        
#     try:
#         type5 = type_string5.replace("\"", "")
#         type5_index = make_map[make5]
#         make_vector[4] = make5_index
#     except Exception as e:
#         print('sku_string={}'.format(make_string5))
#         print("I/O error({})".format(e))
        
#     return make_vector

In [254]:
order_category_df['type_vector'] = order_category_df.apply(lambda x: generate_column_vector([x['TYPE1'], x['TYPE2'], x['TYPE3'], x['TYPE4'], x['TYPE5']], type_map), axis=1)

In [255]:
order_category_df['type_vector'].head()

0    [1, 0, 4, 0, 0]
1    [4, 0, 1, 0, 0]
2    [4, 0, 1, 0, 0]
3    [4, 0, 1, 0, 0]
4    [4, 0, 1, 0, 0]
Name: type_vector, dtype: object

In [256]:
order_category_df.drop(columns=['TYPE1', 'TYPE2', 'TYPE3', 'TYPE4', 'TYPE5'], inplace=True)

In [257]:
def collect_models(model_string1, model_string2, model_string3, model_string4, model_string5):
#     print('type_string1={}'.format(type_string1))
    all_models.add(model_string1)
    all_models.add(model_string2)
    all_models.add(model_string3)
    all_models.add(model_string4)
    all_models.add(model_string5)

In [258]:
all_models = set()
for row in order_category_df.iterrows():
    collect_models(row[1]['MODEL1'], row[1]['MODEL2'], row[1]['MODEL3'], row[1]['MODEL4'], row[1]['MODEL5'], )

In [259]:
all_model_list = list(all_models)

In [260]:
all_model_list

[nan,
 'iPhone 6s Plus',
 'Galaxy A70',
 'iPhone XS',
 'Watch Series 4 Aluminum 40mm',
 'iPhone 8 Plus',
 'iPhone XS Max',
 'Galaxy Note10+',
 'iPad (6th Generation)',
 'iPhone 11 Pro Max',
 'iPhone 8',
 'Galaxy A50',
 'Galaxy Note10',
 'Galaxy S8+ BYOD',
 'Galaxy S10 BYOD',
 'Galaxy S8 BYOD',
 'Galaxy S9 BYOD',
 'Galaxy S9+',
 'Watch Series 3 Aluminum 42mm',
 'Galaxy S10 Unlocked BYOD',
 'Stylo 5',
 'iPhone X',
 'iPhone 7',
 'Galaxy S8',
 'iPhone 8 (PRODUCT)RED - LTE Only',
 'iPad Pro 11-inch Cellular',
 'G8 ThinQTM',
 'iPhone 8 Plus (PRODUCT)RED',
 'Pixel 4',
 'K30&',
 'iPad Air',
 'iPhone SE',
 'iPhone 7 Plus',
 'Galaxy Note9',
 'Galaxy S10e',
 'Galaxy S9+ BYOD',
 'moto e5 play',
 'iPhone 6s',
 'Galaxy Note8 BYOD',
 'iPhone 8 Plus - LTE Only',
 'Watch Series 3 Aluminum 38mm',
 'iPhone 11 Pro',
 'Watch Series 5 Aluminum 40mm',
 'Accessory',
 'Galaxy Note9 BYOD',
 'Watch Series 4 Aluminum 44mm',
 'iPhone X LTE Only',
 'iPhone 6',
 'Galaxy S9',
 'iPhone 8 LTE Only',
 'Galaxy S10+',
 'i

In [261]:
model_map = {}
index = 0          # already accounted for nan
for model in all_model_list:
    model_map[model] = index
    index += 1

In [262]:
model_map

{nan: 0,
 'iPhone 6s Plus': 1,
 'Galaxy A70': 2,
 'iPhone XS': 3,
 'Watch Series 4 Aluminum 40mm': 4,
 'iPhone 8 Plus': 5,
 'iPhone XS Max': 6,
 'Galaxy Note10+': 7,
 'iPad (6th Generation)': 8,
 'iPhone 11 Pro Max': 9,
 'iPhone 8': 10,
 'Galaxy A50': 11,
 'Galaxy Note10': 12,
 'Galaxy S8+ BYOD': 13,
 'Galaxy S10 BYOD': 14,
 'Galaxy S8 BYOD': 15,
 'Galaxy S9 BYOD': 16,
 'Galaxy S9+': 17,
 'Watch Series 3 Aluminum 42mm': 18,
 'Galaxy S10 Unlocked BYOD': 19,
 'Stylo 5': 20,
 'iPhone X': 21,
 'iPhone 7': 22,
 'Galaxy S8': 23,
 'iPhone 8 (PRODUCT)RED - LTE Only': 24,
 'iPad Pro 11-inch Cellular': 25,
 'G8 ThinQTM': 26,
 'iPhone 8 Plus (PRODUCT)RED': 27,
 'Pixel 4': 28,
 'K30&': 29,
 'iPad Air': 30,
 'iPhone SE': 31,
 'iPhone 7 Plus': 32,
 'Galaxy Note9': 33,
 'Galaxy S10e': 34,
 'Galaxy S9+ BYOD': 35,
 'moto e5 play': 36,
 'iPhone 6s': 37,
 'Galaxy Note8 BYOD': 38,
 'iPhone 8 Plus - LTE Only': 39,
 'Watch Series 3 Aluminum 38mm': 40,
 'iPhone 11 Pro': 41,
 'Watch Series 5 Aluminum 40mm': 4

In [263]:
# def generate_model_vector(model_strings):
# #     print('row = {}'.format(row))

#     model_vector = [0] * 5       # 5 types max. 0 means nan

# # #     print('truncated_string={}'.format(truncated_string))
# #     no_skus = len(sku_array)
#     i = 0
#     for model_string in model_strings:
#         try:
#             model = model_string.replace("\"", "")
#             model_index = model_map[model]
#             model_vector[0] = model_index
#         except Exception as e:
#             print('model_string={}'.format(model_string))
#             print("I/O error({})".format(e))
        
#     return model_vector

In [264]:

order_category_df['model_vector'] = order_category_df.apply(lambda x: generate_column_vector([x['MODEL1'], x['MODEL2'], x['MODEL3'], x['MODEL4'], x['MODEL5']], model_map), axis=1)

In [265]:
order_category_df['model_vector'].head()

0    [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, ...
1    [4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
2    [4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
3    [4, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
4    [4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
Name: model_vector, dtype: object

In [266]:
order_category_df.drop(['MODEL1'], axis=1, inplace=True)
order_category_df.drop(['MODEL2'], axis=1, inplace=True)
order_category_df.drop(['MODEL3'], axis=1, inplace=True)
order_category_df.drop(['MODEL4'], axis=1, inplace=True)
order_category_df.drop(['MODEL5'], axis=1, inplace=True)

In [267]:
order_category_df.columns

Index(['EXPERIAN_RESPONSE_1', 'EXPERIAN_RESPONSE_2', 'ORDER_EXCHANGE',
       'TPF_HIGH_YELLOW', 'IDA_RESULT', 'BLACKLIST_EMAIL', 'FRAUDNET_RESULT',
       'YELLOW_SKU', 'SHIPPING_STATE', 'SALES_CHANNEL', 'BILLING_STATE',
       'ACCOUNT_NUMBER', 'IS_EXISTING_CUSTOMER', 'LINE_LIMIT_REACHED',
       'SSN_BLACKLISTED', 'INTERNAL_CREDIT_CHECK_DONE', 'DEVICE_AT_HOME',
       'FPF_HIGH_YELLOW', 'DENIAL_OF_SERVICES', 'SHIPPINGMETHOD',
       'EXTERNAL_CREDIT_CHECK_DONE', 'SKU_vector', 'carrier_vector',
       'make_vector', 'type_vector', 'model_vector'],
      dtype='object')

In [268]:
# we end pre-processing here
# use the transform functions in fastai to convert single value categorical columns

In [269]:
order_category_df.head()

,EXPERIAN_RESPONSE_1,EXPERIAN_RESPONSE_2,ORDER_EXCHANGE,TPF_HIGH_YELLOW,IDA_RESULT,BLACKLIST_EMAIL,FRAUDNET_RESULT,YELLOW_SKU,SHIPPING_STATE,SALES_CHANNEL,...,DEVICE_AT_HOME,FPF_HIGH_YELLOW,DENIAL_OF_SERVICES,SHIPPINGMETHOD,EXTERNAL_CREDIT_CHECK_DONE,SKU_vector,carrier_vector,make_vector,type_vector,model_vector
0,NaN,NaN,NaN,N,GREEN,N,NaN,N,MN,TELESALES,...,N,N,N,2Days,N,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0]","[1, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 2, ...","[1, 0, 4, 0, 0]","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, ..."
1,NaN,NaN,NaN,N,GREEN,N,NaN,N,MN,ONLINE,...,Y,N,N,2Days,N,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[21, 0, 0, 0, 0]","[4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, ...","[4, 0, 1, 0, 0]","[4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,NaN,NaN,NaN,N,GREEN,N,NaN,N,MN,TELESALES,...,N,N,N,2Days,N,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[2, 0, 0, 0, 0]","[4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, ...","[4, 0, 1, 0, 0]","[4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,NaN,NaN,NaN,N,GREEN,N,NaN,N,MN,TELESALES,...,N,N,N,2Days,Y,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0]","[4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, ...","[4, 0, 1, 0, 0]","[4, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,NaN,NaN,NaN,N,GREEN,N,NaN,N,MN,ONLINE,...,Y,N,N,2Days,N,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[13, 0, 0, 0, 0]","[4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, ...","[4, 0, 1, 0, 0]","[4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [270]:
order_category_df.to_csv('ivr_backup_data/order_category_df.csv', index=False)

In [271]:
order_category_df.columns

Index(['EXPERIAN_RESPONSE_1', 'EXPERIAN_RESPONSE_2', 'ORDER_EXCHANGE',
       'TPF_HIGH_YELLOW', 'IDA_RESULT', 'BLACKLIST_EMAIL', 'FRAUDNET_RESULT',
       'YELLOW_SKU', 'SHIPPING_STATE', 'SALES_CHANNEL', 'BILLING_STATE',
       'ACCOUNT_NUMBER', 'IS_EXISTING_CUSTOMER', 'LINE_LIMIT_REACHED',
       'SSN_BLACKLISTED', 'INTERNAL_CREDIT_CHECK_DONE', 'DEVICE_AT_HOME',
       'FPF_HIGH_YELLOW', 'DENIAL_OF_SERVICES', 'SHIPPINGMETHOD',
       'EXTERNAL_CREDIT_CHECK_DONE', 'SKU_vector', 'carrier_vector',
       'make_vector', 'type_vector', 'model_vector'],
      dtype='object')

In [272]:
order_category_df.shape

(244154, 26)

In [124]:
NaN_cat_list = order_category_df.columns[order_category_df.isna().any()].tolist()

In [125]:
NaN_cat_list

['EXPERIAN_RESPONSE_1',
 'EXPERIAN_RESPONSE_2',
 'ORDER_EXCHANGE',
 'IDA_RESULT',
 'FRAUDNET_RESULT',
 'SHIPPING_STATE',
 'BILLING_STATE',
 'INTERNAL_CREDIT_CHECK_DONE',
 'SHIPPINGMETHOD']

In [126]:
state_list = order_category_df['SHIPPING_STATE'].unique()

In [127]:
state_list

array(['MN', 'MA', 'WI', 'CA', 'NH', nan, 'ME', 'CT', 'VT', 'TX', 'NY',
       'OR', 'WA', 'PA', 'WV', 'MD', 'OH', 'GA', 'FL', 'MI', 'DE', 'VA',
       'DC', 'SC', 'LA', 'TN', 'MS', 'AR', 'KY', 'AL', 'IN', 'UT', 'NC',
       'AZ', 'CO', 'IL', 'NM', 'NJ', 'MO', 'KS', 'ID'], dtype=object)

In [128]:
len(state_list)

41

In [ ]:
# only 41 states show up in data. This is potentially a problem using fastai lib which only use categorical values present 
# in the data
# if a new state show up in the future, may bomb out -> requires new training

In [127]:
order_category_df.dropna(subset=['SHIPPING_STATE'], inplace=True)

In [128]:
order_category_df['SHIPPING_STATE'].unique()

array(['MN', 'MA', 'WI', 'CA', 'NH', 'ME', 'CT', 'VT', 'TX', 'NY', 'OR',
       'WA', 'PA', 'WV', 'MD', 'OH', 'GA', 'FL', 'MI', 'DE', 'VA', 'DC',
       'SC', 'LA', 'TN', 'MS', 'AR', 'KY', 'AL', 'IN', 'UT', 'NC', 'AZ',
       'CO', 'IL', 'NM', 'NJ', 'MO', 'KS', 'ID'], dtype=object)

In [133]:
state_list = [line.rstrip('\n') for line in open('ivr_data/states.txt')]

In [134]:
len(state_list)

51

In [135]:
state_map = {}
index = 0          # already accounted for nan
for state in state_list:
    state_map[state] = index
    index += 1

In [136]:
# state_map

In [137]:
order_category_df['SHIPPING_STATE_cat'] = order_category_df.apply(lambda x: generate_column_index(x['SHIPPING_STATE'], state_map), axis=1)

In [138]:
order_category_df.drop(['SHIPPING_STATE'], axis=1, inplace=True)

In [139]:
order_category_df.dropna(subset=['BILLING_STATE'], inplace=True)

In [140]:
# order_category_df[order_category_df['BILLING_STATE'].isna()].shape

In [141]:
order_category_df['BILLING_STATE_cat'] = order_category_df.apply(lambda x: generate_column_index(x['BILLING_STATE'], state_map), axis=1)

In [142]:
# order_category_df['BILLING_STATE_vector'].head()

In [143]:
# order_category_df.iloc[100]['BILLING_STATE_vector']

In [144]:
order_category_df.drop(['BILLING_STATE'], axis=1, inplace=True)

In [145]:
# order_category_df.columns

Go down remaining single categorical fields, one at time

In [146]:
order_category_df.columns

Index(['IS_EXISTING_CUSTOMER', 'FPF_HIGH_YELLOW', 'LINE_LIMIT_REACHED',
       'IDA_RESULT', 'YELLOW_SKU', 'DEVICE_AT_HOME', 'ORDER_EXCHANGE',
       'SHIPPINGMETHOD', 'EXPERIAN_RESPONSE_2', 'SSN_BLACKLISTED',
       'EXPERIAN_RESPONSE_1', 'BLACKLIST_EMAIL', 'INTERNAL_CREDIT_CHECK_DONE',
       'DENIAL_OF_SERVICES', 'FRAUDNET_RESULT', 'TPF_HIGH_YELLOW',
       'EXTERNAL_CREDIT_CHECK_DONE', 'SALES_CHANNEL', 'SKU_vector',
       'carrier_vector', 'make_vector', 'type_vector', 'model_vector',
       'SHIPPING_STATE_cat', 'BILLING_STATE_cat'],
      dtype='object')

In [147]:
order_category_df['LINE_LIMIT_REACHED'].unique()

array(['N', 'Y'], dtype=object)

In [148]:
binary_map = {'N': 0, 'Y': 1}

In [149]:
def convert_binary(YN_string):
    return binary_map[YN_string]

In [150]:
order_category_df['LINE_LIMIT_REACHED_cat'] = order_category_df['LINE_LIMIT_REACHED'].apply(lambda x : convert_binary(x))

In [151]:
order_category_df['LINE_LIMIT_REACHED_cat'].unique()

array([0, 1])

In [152]:
order_category_df.drop(['LINE_LIMIT_REACHED'], axis=1, inplace=True)

In [153]:
order_category_df.columns

Index(['IS_EXISTING_CUSTOMER', 'FPF_HIGH_YELLOW', 'IDA_RESULT', 'YELLOW_SKU',
       'DEVICE_AT_HOME', 'ORDER_EXCHANGE', 'SHIPPINGMETHOD',
       'EXPERIAN_RESPONSE_2', 'SSN_BLACKLISTED', 'EXPERIAN_RESPONSE_1',
       'BLACKLIST_EMAIL', 'INTERNAL_CREDIT_CHECK_DONE', 'DENIAL_OF_SERVICES',
       'FRAUDNET_RESULT', 'TPF_HIGH_YELLOW', 'EXTERNAL_CREDIT_CHECK_DONE',
       'SALES_CHANNEL', 'SKU_vector', 'carrier_vector', 'make_vector',
       'type_vector', 'model_vector', 'SHIPPING_STATE_cat',
       'BILLING_STATE_cat', 'LINE_LIMIT_REACHED_cat'],
      dtype='object')

In [154]:
order_category_df['SSN_BLACKLISTED'].unique()

array(['N', 'Y'], dtype=object)

In [155]:
order_category_df['SSN_BLACKLISTED_cat'] = order_category_df['SSN_BLACKLISTED'].apply(lambda x : convert_binary(x))

In [156]:
order_category_df['SSN_BLACKLISTED_cat'].unique()

array([0, 1])

In [157]:
order_category_df.drop(['SSN_BLACKLISTED'], axis=1, inplace=True)

In [158]:
order_category_df.columns

Index(['IS_EXISTING_CUSTOMER', 'FPF_HIGH_YELLOW', 'IDA_RESULT', 'YELLOW_SKU',
       'DEVICE_AT_HOME', 'ORDER_EXCHANGE', 'SHIPPINGMETHOD',
       'EXPERIAN_RESPONSE_2', 'EXPERIAN_RESPONSE_1', 'BLACKLIST_EMAIL',
       'INTERNAL_CREDIT_CHECK_DONE', 'DENIAL_OF_SERVICES', 'FRAUDNET_RESULT',
       'TPF_HIGH_YELLOW', 'EXTERNAL_CREDIT_CHECK_DONE', 'SALES_CHANNEL',
       'SKU_vector', 'carrier_vector', 'make_vector', 'type_vector',
       'model_vector', 'SHIPPING_STATE_cat', 'BILLING_STATE_cat',
       'LINE_LIMIT_REACHED_cat', 'SSN_BLACKLISTED_cat'],
      dtype='object')

In [186]:
def generate_column_index(column_string, column_map):
    len_map = len(column_map)
#     print('row = {}'.format(row))

    column_index = 0       # 0 means null

    try:
#             print('column={}'.format(column))
        column_index = column_map[column_string]
#         print('column_string = {}, column_index={}'.format(column_string, column_index))
    except Exception as e:
        print('column_string={}'.format(column_string))
        print("I/O error({})".format(e))
        
    return column_index

In [174]:
EXPERIAN_RESPONSE_1_list = order_category_df['EXPERIAN_RESPONSE_1'].unique().tolist()

In [175]:
EXPERIAN_RESPONSE_1_list

[nan,
 '3100/ACCEPT/Continue/ACCEPT- FraudNet only',
 '3100/REFER/Continue and Investigate/Suspicious fraud device indicators - medium',
 '3100/REFER/Continue and Investigate/Suspicious fraud device indicators - high',
 '3100/NODECISION/No Decision/Error Or No Decision From FS']

In [176]:
EXPERIAN_RESPONSE_1_map = {}

i = 0
for response_item in EXPERIAN_RESPONSE_1_list:
    EXPERIAN_RESPONSE_1_map[response_item] = i
    i += 1

In [177]:
EXPERIAN_RESPONSE_1_map

{nan: 0,
 '3100/ACCEPT/Continue/ACCEPT- FraudNet only': 1,
 '3100/REFER/Continue and Investigate/Suspicious fraud device indicators - medium': 2,
 '3100/REFER/Continue and Investigate/Suspicious fraud device indicators - high': 3,
 '3100/NODECISION/No Decision/Error Or No Decision From FS': 4}

In [187]:
order_category_df['EXPERIAN_RESPONSE_1_cat'] = order_category_df.apply(lambda x: generate_column_index(x['EXPERIAN_RESPONSE_1'], EXPERIAN_RESPONSE_1_map), axis=1)

In [189]:
order_category_df['EXPERIAN_RESPONSE_1_cat'].head(20)

0     0
1     0
2     0
3     0
4     0
5     0
6     0
7     1
8     0
9     0
10    1
11    1
12    1
13    1
14    0
15    1
16    0
17    1
18    1
19    0
Name: EXPERIAN_RESPONSE_1_cat, dtype: int64

In [190]:
order_category_df['EXPERIAN_RESPONSE_1'].head(20)

0                                            NaN
1                                            NaN
2                                            NaN
3                                            NaN
4                                            NaN
5                                            NaN
6                                            NaN
7     3100/ACCEPT/Continue/ACCEPT- FraudNet only
8                                            NaN
9                                            NaN
10    3100/ACCEPT/Continue/ACCEPT- FraudNet only
11    3100/ACCEPT/Continue/ACCEPT- FraudNet only
12    3100/ACCEPT/Continue/ACCEPT- FraudNet only
13    3100/ACCEPT/Continue/ACCEPT- FraudNet only
14                                           NaN
15    3100/ACCEPT/Continue/ACCEPT- FraudNet only
16                                           NaN
17    3100/ACCEPT/Continue/ACCEPT- FraudNet only
18    3100/ACCEPT/Continue/ACCEPT- FraudNet only
19                                           NaN
Name: EXPERIAN_RESPO

In [191]:
order_category_df.drop(['EXPERIAN_RESPONSE_1'], axis=1, inplace=True)

In [192]:
order_category_df.columns

Index(['IS_EXISTING_CUSTOMER', 'FPF_HIGH_YELLOW', 'IDA_RESULT', 'YELLOW_SKU',
       'DEVICE_AT_HOME', 'ORDER_EXCHANGE', 'SHIPPINGMETHOD',
       'EXPERIAN_RESPONSE_2', 'BLACKLIST_EMAIL', 'INTERNAL_CREDIT_CHECK_DONE',
       'DENIAL_OF_SERVICES', 'FRAUDNET_RESULT', 'TPF_HIGH_YELLOW',
       'EXTERNAL_CREDIT_CHECK_DONE', 'SALES_CHANNEL', 'SKU_vector',
       'carrier_vector', 'make_vector', 'type_vector', 'model_vector',
       'SHIPPING_STATE_cat', 'BILLING_STATE_cat', 'LINE_LIMIT_REACHED_cat',
       'SSN_BLACKLISTED_cat', 'EXPERIAN_RESPONSE_1_cat'],
      dtype='object')

In [193]:
EXPERIAN_RESPONSE_2_list = order_category_df['EXPERIAN_RESPONSE_2'].unique().tolist()

In [194]:
EXPERIAN_RESPONSE_2_list

[nan,
 '3200/APPROVE/700/No Fraud Detected/No device concerns',
 '3200/APPROVE/200/No Fraud Detected/No device concerns',
 '3200/APPROVE/600/No Fraud Detected/No device concerns',
 '3200/REFER-MED/1700/Potential Fraud - Medium Risk/Suspicious fraud device indicators - medium',
 '3200/APPROVE/350/No Fraud Detected/No device concerns',
 '3200/APPROVE/450/No Fraud Detected/No device concerns',
 '3200/APPROVE/1250/No Fraud Detected/No device concerns',
 '3200/APPROVE/750/No Fraud Detected/No device concerns',
 '3200/REFER-MED/1800/Potential Fraud - Medium Risk/Suspicious fraud device indicators - medium',
 '3200/APPROVE/250/No Fraud Detected/No device concerns',
 '3200/APPROVE/1200/No Fraud Detected/No device concerns',
 '3200/APPROVE/550/No Fraud Detected/No device concerns',
 '3200/APPROVE/900/No Fraud Detected/No device concerns',
 '3200/REFER-MED/1850/Potential Fraud - Medium Risk/Suspicious fraud device indicators - medium',
 '3200/APPROVE/300/No Fraud Detected/No device concerns',
 '

In [195]:
EXPERIAN_RESPONSE_2_map = {}

i = 0
for response_item in EXPERIAN_RESPONSE_2_list:
    EXPERIAN_RESPONSE_2_map[response_item] = i
    i += 1

In [196]:
EXPERIAN_RESPONSE_2_map

{nan: 0,
 '3200/APPROVE/700/No Fraud Detected/No device concerns': 1,
 '3200/APPROVE/200/No Fraud Detected/No device concerns': 2,
 '3200/APPROVE/600/No Fraud Detected/No device concerns': 3,
 '3200/REFER-MED/1700/Potential Fraud - Medium Risk/Suspicious fraud device indicators - medium': 4,
 '3200/APPROVE/350/No Fraud Detected/No device concerns': 5,
 '3200/APPROVE/450/No Fraud Detected/No device concerns': 6,
 '3200/APPROVE/1250/No Fraud Detected/No device concerns': 7,
 '3200/APPROVE/750/No Fraud Detected/No device concerns': 8,
 '3200/REFER-MED/1800/Potential Fraud - Medium Risk/Suspicious fraud device indicators - medium': 9,
 '3200/APPROVE/250/No Fraud Detected/No device concerns': 10,
 '3200/APPROVE/1200/No Fraud Detected/No device concerns': 11,
 '3200/APPROVE/550/No Fraud Detected/No device concerns': 12,
 '3200/APPROVE/900/No Fraud Detected/No device concerns': 13,
 '3200/REFER-MED/1850/Potential Fraud - Medium Risk/Suspicious fraud device indicators - medium': 14,
 '3200/APP

In [197]:
order_category_df['EXPERIAN_RESPONSE_2_cat'] = order_category_df.apply(lambda x: generate_column_index(x['EXPERIAN_RESPONSE_2'], EXPERIAN_RESPONSE_2_map), axis=1)

In [198]:
order_category_df['EXPERIAN_RESPONSE_2_cat'].head(20)

0     0
1     0
2     0
3     0
4     0
5     0
6     0
7     1
8     0
9     0
10    2
11    2
12    2
13    2
14    0
15    2
16    0
17    2
18    2
19    0
Name: EXPERIAN_RESPONSE_2_cat, dtype: int64

In [199]:
order_category_df.drop(['EXPERIAN_RESPONSE_2'], axis=1, inplace=True)

In [200]:
order_category_df.columns

Index(['IS_EXISTING_CUSTOMER', 'FPF_HIGH_YELLOW', 'IDA_RESULT', 'YELLOW_SKU',
       'DEVICE_AT_HOME', 'ORDER_EXCHANGE', 'SHIPPINGMETHOD', 'BLACKLIST_EMAIL',
       'INTERNAL_CREDIT_CHECK_DONE', 'DENIAL_OF_SERVICES', 'FRAUDNET_RESULT',
       'TPF_HIGH_YELLOW', 'EXTERNAL_CREDIT_CHECK_DONE', 'SALES_CHANNEL',
       'SKU_vector', 'carrier_vector', 'make_vector', 'type_vector',
       'model_vector', 'SHIPPING_STATE_cat', 'BILLING_STATE_cat',
       'LINE_LIMIT_REACHED_cat', 'SSN_BLACKLISTED_cat',
       'EXPERIAN_RESPONSE_1_cat', 'EXPERIAN_RESPONSE_2_cat'],
      dtype='object')

In [201]:
def generate_cat_field(single_cat_field_name):
    cat_field_name = single_cat_field_name + '_cat'
    unique_value_list = order_category_df[single_cat_field_name].unique().tolist()
    field_map = {}
    i = 0
    for field_item in unique_value_list:
        field_map[field_item] = i
        i += 1
    order_category_df[cat_field_name] = order_category_df.apply(lambda x: generate_column_index(x[single_cat_field_name], field_map), axis=1)    
    

In [203]:
generate_cat_field('IDA_RESULT')

In [204]:
order_category_df['IDA_RESULT_cat'].head(20)

0     0
1     0
2     0
3     0
4     0
5     0
6     0
7     0
8     0
9     0
10    0
11    0
12    0
13    0
14    0
15    0
16    0
17    1
18    0
19    0
Name: IDA_RESULT_cat, dtype: int64

In [205]:
order_category_df.drop(['IDA_RESULT'], axis=1, inplace=True)

In [206]:
order_category_df.columns

Index(['IS_EXISTING_CUSTOMER', 'FPF_HIGH_YELLOW', 'YELLOW_SKU',
       'DEVICE_AT_HOME', 'ORDER_EXCHANGE', 'SHIPPINGMETHOD', 'BLACKLIST_EMAIL',
       'INTERNAL_CREDIT_CHECK_DONE', 'DENIAL_OF_SERVICES', 'FRAUDNET_RESULT',
       'TPF_HIGH_YELLOW', 'EXTERNAL_CREDIT_CHECK_DONE', 'SALES_CHANNEL',
       'SKU_vector', 'carrier_vector', 'make_vector', 'type_vector',
       'model_vector', 'SHIPPING_STATE_cat', 'BILLING_STATE_cat',
       'LINE_LIMIT_REACHED_cat', 'SSN_BLACKLISTED_cat',
       'EXPERIAN_RESPONSE_1_cat', 'EXPERIAN_RESPONSE_2_cat', 'IDA_RESULT_cat'],
      dtype='object')

In [207]:
generate_cat_field('INTERNAL_CREDIT_CHECK_DONE')
order_category_df.drop(['INTERNAL_CREDIT_CHECK_DONE'], axis=1, inplace=True)

In [208]:
generate_cat_field('TPF_HIGH_YELLOW')
order_category_df.drop(['TPF_HIGH_YELLOW'], axis=1, inplace=True)

In [209]:
generate_cat_field('EXTERNAL_CREDIT_CHECK_DONE')
order_category_df.drop(['EXTERNAL_CREDIT_CHECK_DONE'], axis=1, inplace=True)

In [210]:
generate_cat_field('IS_EXISTING_CUSTOMER')
order_category_df.drop(['IS_EXISTING_CUSTOMER'], axis=1, inplace=True)

In [211]:
generate_cat_field('SALES_CHANNEL')
order_category_df.drop(['SALES_CHANNEL'], axis=1, inplace=True)

In [212]:
generate_cat_field('SHIPPINGMETHOD')
order_category_df.drop(['SHIPPINGMETHOD'], axis=1, inplace=True)

In [213]:
generate_cat_field('FPF_HIGH_YELLOW')
order_category_df.drop(['FPF_HIGH_YELLOW'], axis=1, inplace=True)

In [214]:
generate_cat_field('FRAUDNET_RESULT')
order_category_df.drop(['FRAUDNET_RESULT'], axis=1, inplace=True)

In [215]:
generate_cat_field('YELLOW_SKU')
order_category_df.drop(['YELLOW_SKU'], axis=1, inplace=True)

In [216]:
order_category_df['DEVICE_AT_HOME'].head()

0    N
1    Y
2    N
3    N
4    Y
Name: DEVICE_AT_HOME, dtype: object

In [217]:
generate_cat_field('DEVICE_AT_HOME')
order_category_df.drop(['DEVICE_AT_HOME'], axis=1, inplace=True)

In [218]:
# BLACKLIST_EMAIL is too sparse, drop
order_category_df.drop(['BLACKLIST_EMAIL'], axis=1, inplace=True)

In [219]:
generate_cat_field('ORDER_EXCHANGE')
order_category_df.drop(['ORDER_EXCHANGE'], axis=1, inplace=True)

In [220]:
generate_cat_field('DENIAL_OF_SERVICES')
order_category_df.drop(['DENIAL_OF_SERVICES'], axis=1, inplace=True)

In [221]:
order_category_df.columns

Index(['SKU_vector', 'carrier_vector', 'make_vector', 'type_vector',
       'model_vector', 'SHIPPING_STATE_cat', 'BILLING_STATE_cat',
       'LINE_LIMIT_REACHED_cat', 'SSN_BLACKLISTED_cat',
       'EXPERIAN_RESPONSE_1_cat', 'EXPERIAN_RESPONSE_2_cat', 'IDA_RESULT_cat',
       'INTERNAL_CREDIT_CHECK_DONE_cat', 'TPF_HIGH_YELLOW_cat',
       'EXTERNAL_CREDIT_CHECK_DONE_cat', 'IS_EXISTING_CUSTOMER_cat',
       'SALES_CHANNEL_cat', 'SHIPPINGMETHOD_cat', 'FPF_HIGH_YELLOW_cat',
       'FRAUDNET_RESULT_cat', 'YELLOW_SKU_cat', 'DEVICE_AT_HOME_cat',
       'ORDER_EXCHANGE_cat', 'DENIAL_OF_SERVICES_cat'],
      dtype='object')

Only vector fields left. conversion is complete. save results

In [222]:
order_category_df.to_csv('ivr_backup_data/order_category_df.csv', index=False)

In [223]:
order_by_account = order_extract_df.groupby('ACCOUNT_NUMBER')

In [224]:
order_by_account.size().unique()

array([ 1,  4,  2,  3,  9,  6,  5,  7,  8, 10, 12, 11, 13, 16, 14, 22, 20,
       19, 49, 26, 33, 15, 23, 21, 17])

In [225]:
# i = 1
# for group_name, df_group in order_by_account:
#     print('group name={}'.format(group_name))
#     print('group size={}'.format(df_group.size))
#     i += 1
#     if i > 1:
#         break

In [226]:
group_count_df = order_by_account.size().reset_index(name='counts')

In [227]:
group_count_df[group_count_df['counts']==49]

,ACCOUNT_NUMBER,counts
100636,2173973859,49


In [228]:
min(order_extract_df['ORDER_CREATED_DATE'])

'2019-04-30 00:00:22'

In [229]:
max(order_extract_df['ORDER_CREATED_DATE'])

'2019-11-30 23:59:04'

In [230]:
order_by_account.ngroups

198233

In [231]:
order_category_df.columns

Index(['SKU_vector', 'carrier_vector', 'make_vector', 'type_vector',
       'model_vector', 'SHIPPING_STATE_cat', 'BILLING_STATE_cat',
       'LINE_LIMIT_REACHED_cat', 'SSN_BLACKLISTED_cat',
       'EXPERIAN_RESPONSE_1_cat', 'EXPERIAN_RESPONSE_2_cat', 'IDA_RESULT_cat',
       'INTERNAL_CREDIT_CHECK_DONE_cat', 'TPF_HIGH_YELLOW_cat',
       'EXTERNAL_CREDIT_CHECK_DONE_cat', 'IS_EXISTING_CUSTOMER_cat',
       'SALES_CHANNEL_cat', 'SHIPPINGMETHOD_cat', 'FPF_HIGH_YELLOW_cat',
       'FRAUDNET_RESULT_cat', 'YELLOW_SKU_cat', 'DEVICE_AT_HOME_cat',
       'ORDER_EXCHANGE_cat', 'DENIAL_OF_SERVICES_cat'],
      dtype='object')